In [32]:
import scanpy as sc
import numpy as np 
import pandas as pd
# openpyxl package to read excel# 'pip install openpyxl' if undownload it

In [55]:
supplementaryTable = "Supplementary table.xlsx"
dataset = "PBMC_scPortal"
dataset_url = "https://atlas.fredhutch.org/data/nygc/multimodal/pbmc_multimodal.h5seurat"
CellType_ColumnName = 'CellType'

In [4]:
# sign up and download by command (bulk download), url:
# https://singlecell.broadinstitute.org/single_cell/study/SCP424/single-cell-comparison-pbmc-data#study-download
# this command is timeliness

/bin/bash: -c: line 0: syntax error near unexpected token `{'
/bin/bash: -c: line 0: `curl.exe "https://singlecell.broadinstitute.org/single_cell/api/v1/bulk_download/generate_curl_config?accessions=SCP424&auth_code=Ogn5Et6Z&directory=all&context=study"  -o cfg.txt; curl.exe -K cfg.txt ;if ($?) { rm cfg.txt }'


In [26]:
!curl -k "https://singlecell.broadinstitute.org/single_cell/api/v1/bulk_download/generate_curl_config?accessions=SCP424&auth_code=Qjcvca42&directory=all&context=study"  -o cfg.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21191    0 21191    0     0   9751      0 --:--:--  0:00:02 --:--:--  9747


In [27]:
!curl -K cfg.txt ;

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5828  100  5828    0     0   9585      0 --:--:-- --:--:-- --:--:--  9569
100  145M  100  145M    0     0  41.6M      0  0:00:03  0:00:03 --:--:-- 59.1M
100 1109k  100 1109k    0     0  1849k      0 --:--:-- --:--:-- --:--:-- 20.7M
100 79238  100 79238    0     0   140k      0 --:--:-- --:--:-- --:--:--  140k
100  5426  100  5426    0     0  10161      0 --:--:-- --:--:-- --:--:--     0
100  339k  100  339k    0     0   589k      0 --:--:-- --:--:-- --:--:--  589k
100  233k  100  233k    0     0   422k      0 --:--:-- --:--:-- --:--:-- 14.0M
100  109M  100  109M    0     0  36.4M      0  0:00:03  0:00:03 --:--:-- 47.3M
100  220k  100  220k    0     0   388k      0 --:--:-- --:--:-- --:--:--  388k
100  340k  100  340k    0     0   651k      0 --:--:-- --:--:-- --:--:-- 13.5M
100 82179  100 82179    0     0   144k      0 --:--:

In [28]:
!rm cfg.txt

In [35]:
barcode_path = 'SCP424/expression/5cf01beb328cee0dabcf09ee/barcodes.txt'
gene_path = 'SCP424/expression/5cf01beb328cee0dabcf09ee/genes.txt'
matrix_path = 'SCP424/expression/5cf01beb328cee0dabcf09ee/gene_sorted-expression.txt'
meta_path = 'SCP424/metadata/meta.txt'

In [45]:
from scipy import sparse
barcode = pd.read_table(barcode_path,header=None,names=["barcode"])
gene = pd.read_table(gene_path,header=None,names=["ensembl_ids","gene"])
m = pd.read_table(matrix_path,sep=" ").values
matrix = sparse.coo_matrix(
    (m[1:,2], (m[1:,1].astype(np.int32)-1,
    m[1:,0].astype(np.int32)-1)),
    shape=(barcode.shape[0],gene.shape[0])
)

In [47]:
adata = sc.AnnData(
    X = matrix.tocsc(),
    var = gene,
    obs = barcode
    )
select = [("10X_V2" in bar)|("10x_v2" in bar) for bar in adata.obs["barcode"]]
adata = adata[select,:]
adata 

/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


View of AnnData object with n_obs × n_vars = 9806 × 33694
    obs: 'barcode'
    var: 'ensembl_ids', 'gene'

## Get label

In [49]:
metadata = pd.read_table(meta_path)
metadata = metadata[metadata["Method"].isin(["10x Chromium (v2)","10x Chromium (v2) A","10x Chromium (v2) B"])]

In [50]:
metadata

,NAME,nGene,nUMI,percent.mito,Cluster,CellType,Experiment,Method
507,pbmc1_10x_v2_A_AAAGATGCAAAGTCAA,851,2177,0.0381258612769867,5,CD14+ monocyte,pbmc1,10x Chromium (v2) A
508,pbmc1_10x_v2_A_AAAGCAAGTAGGAGTC,1078,3065,0.0417618270799347,5,CD14+ monocyte,pbmc1,10x Chromium (v2) A
509,pbmc1_10x_v2_A_AAAGCAATCGGTTCGG,538,977,0.0992835209825998,4,CD14+ monocyte,pbmc1,10x Chromium (v2) A
510,pbmc1_10x_v2_A_AAAGTAGTCATTTGGG,1544,4933,0.0427731603486722,5,CD14+ monocyte,pbmc1,10x Chromium (v2) A
511,pbmc1_10x_v2_A_AAAGTAGTCCGAGCCA,632,1487,0.0477471418964358,4,CD14+ monocyte,pbmc1,10x Chromium (v2) A
...,...,...,...,...,...,...,...,...
23742,pbmc2_10X_V2_TTTGTCACACGTCTCT,1266,3142,0.0257797581158498,2,B cell,pbmc2,10x Chromium (v2)
23743,pbmc2_10X_V2_TTTGTCACAGCCAATT,1229,2497,0.0504605526631958,1,Cytotoxic T cell,pbmc2,10x Chromium (v2)
23744,pbmc2_10X_V2_TTTGTCAGTACGCACC,19,20,0,0,CD4+ T cell,pbmc2,10x Chromium (v2)
23745,pbmc2_10X_V2_TTTGTCATCAGTACGT,997,2640,0.0212121212121212,2,B cell,pbmc2,10x Chromium (v2)


In [62]:
df = pd.read_excel(supplementaryTable, sheet_name=dataset)
adata.obs[CellType_ColumnName] = metadata["CellType"].values

In [64]:
def add_celltype(adata,celltype_columnname):
    adata.obs[celltype_columnname] = adata.obs[celltype_columnname].astype("category")
    Transferlist = [np.where(df['origin cell type']==CellTypeList)[0][0] for CellTypeList in adata.obs[celltype_columnname].values.categories]
    adata.obs['modelA id'] = df['modelA id'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
    adata.obs['modelC id'] = df['modelC id'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
    adata.obs['transfer cell type'] = df['transfer cell type'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
add_celltype(adata,CellType_ColumnName)

In [ ]:
adata.obs['origin cell type'] = adata.obs['CellType']

In [65]:
adata.obs

,barcode,CellType,modelA id,modelC id,transfer cell type
0,pbmc1_10x_v2_A_AAAGATGCAAAGTCAA,CD14+ monocyte,10,8,monocyte
1,pbmc1_10x_v2_A_AAAGCAAGTAGGAGTC,CD14+ monocyte,10,8,monocyte
2,pbmc1_10x_v2_A_AAAGCAATCGGTTCGG,CD14+ monocyte,10,8,monocyte
3,pbmc1_10x_v2_A_AAAGTAGTCATTTGGG,CD14+ monocyte,10,8,monocyte
4,pbmc1_10x_v2_A_AAAGTAGTCCGAGCCA,CD14+ monocyte,10,8,monocyte
...,...,...,...,...,...
23741,pbmc2_10X_V2_TTTGTCACACGTCTCT,B cell,3,3,naive B cell
23742,pbmc2_10X_V2_TTTGTCACAGCCAATT,Cytotoxic T cell,1,1,cytotoxic T cell
23743,pbmc2_10X_V2_TTTGTCAGTACGCACC,CD4+ T cell,0,0,T-helper cell
23744,pbmc2_10X_V2_TTTGTCATCAGTACGT,B cell,3,3,naive B cell


In [66]:
adata.write_h5ad(f'../{dataset}.h5ad')

/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'transfer cell type' as categorical


In [ ]:
# !rm -r SCP424